In [ ]:
## Calculate profiles

%reload_ext autoreload
%autoreload 2

from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import colormaps as cm
from matplotlib import colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from lib.Derived import Derived
from functions.plot_data import plot_data

In [ ]:
# Check that the Stokes object file exists
stokes_filename = "generated/objects/stokes.pickle"
my_file = Path(stokes_filename)
if not my_file.is_file():
    print("Stokes object does not exist (", stokes_filename,"), please run file init_data.py first")

# Check that the Nova object file exists
nova_filename = "generated/objects/derived.pickle"
my_file = Path(nova_filename)
if not my_file.is_file():
    print("Nova object does not exist (", nova_filename,"), please run file init_data.py first")

# Load the objects from the files
print("Loading Stokes data from file", stokes_filename)
stokes_list = pickle.load(open(stokes_filename, "rb"))
print("Loading Nova data from file", nova_filename)
nova = pickle.load(open(nova_filename, "rb"))

In [ ]:
# Select pixels of interest
x_pix = np.array([500, 180, 400, 290, 350, 300])
y_pix = np.array([650, 655, 500, 230, 320, 355])

# Plot polarization and selected points
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 8))
ax.scatter(x_pix, y_pix, facecolors='none', edgecolors='red')
ax.set_xlabel('X axis (array index)')
ax.set_ylabel('Y axis (array index)')
img = ax.imshow(nova.mp, cmap='gray')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Number of counts')
plt.show()

In [ ]:
# Plot profiles of each pixel
line_wavelengths = [6301.5015, 6302.4893]
titles = [f'Quiet sun', f'Exterior to  sunspot', f'Penumbra 1', f'Penmbra 2', f'Umbra 1', f'Umbra 2']
fig, axs = plt.subplots(3, 2, figsize=(12,12))
fig.suptitle('Stokes parameter variations over all wavelengths', fontsize=20)
for param in stokes_list:

    for i in range(3):
        for j in range(2):
            axs[i, j].plot(stokes_list[param].wave_array, stokes_list[param].data_n[y_pix[i*2+j],x_pix[i*2+j],:], linestyle='-', label=param)
            axs[i, j].set_title(titles[i*2+j])
            axs[i, j].vlines(line_wavelengths, stokes_list[param].data_n[y_pix[i*2+j],x_pix[i*2+j],:].min(), stokes_list[param].data_n[y_pix[i*2+j],x_pix[i*2+j],:].max(), colors='gray', linestyles='dashed')
            # axs[i, j].legend()


axs[0, 0].legend()

In [ ]:
# Plot profiles of each pixel
pixel_titles = [f'Quiet sun', f'Exterior to  sunspot', f'Penumbra 1', f'Penmbra 2', f'Umbra 1', f'Umbra 2']
fig, axs = plt.subplots(2, 2, figsize=(12,12))
fig.suptitle('Profile variation for selected pixels', fontsize=20)

# Choose a colormap
num_lines = 6
cmap = plt.get_cmap('turbo')
colors = [cmap(i / num_lines) for i in range(num_lines)]

i=0
for param in stokes_list:
    row, col = divmod(i, 2)
    for j in range(num_lines):
        axs[row, col].plot(stokes_list[param].wave_array, stokes_list[param].data_n[y_pix[j],x_pix[j],:], linestyle='-', label=pixel_titles[j], color=colors[j])
        axs[row, col].set_title(param)
        # axs[row, col].vlines(line_wavelengths, stokes_list[param].data_n[y_pix[j],x_pix[j],:].min(), stokes_list[param].data_n[y_pix[j],x_pix[j],:].max(), colors='gray', linestyles='dashed')
        # axs[i, j].legend()

    i += 1


axs[1, 0].legend()
    